In [1]:
# Assumptions
## Assumption 1: Each node has a unique addresses that can be used for lexicographical sorting.
## Assumption 2: Each participating peer / node / actor has a full view of the network (i.e. list of all unique addresses is available to each node)

# Definitions
## Definition 1: Layer: the inverse of a tree height (max at the root and 0 at a leaf)
## Definition 2: Max # of layers: tree height 
## Definition 3: Global Address Book: list of all unique addresses in the network
## Definition 4: Partial Address Book: partial list of all unique addresses nodes in the network (either due to algorithm or due to lack of information)

# Structure
## RainTree: A ternary tree where one of the children of each non-leaf node is a copy of itself with a different subtree
## AddrBook: A sorted list of unique addresses that a node / peer is operating on at a given point in time in the algorithm

# Params
## Let X be the % used to calculate the index of the 1st message sent by a node in `AddrBook` defined above (e.g. X = 1/3 when AddrBook has 9 nodes implies the 3rd node in the sorted list)
## Let Y be the % used to calculate the index of the 2nd message sent by a node in `AddrBook` defined above (e.g. Y = 2/3 when AddrBook has 9 nodes implies the 6th node in the sorted list)
## Let Z be the % by which the `AddrBook` defined above is shrunk with each message propagation (e.g. Z = 2/3 when AddrBook has 9 nodes implies a new AddrBook including nodes 1 through 6)


# Questions to answer:
# Q: What should the stopping condition be aside from cheap log3(N)?
# Q: Why does log3 base achieve the desired height?
# Q: Do X & Y need to be constants?
# Q: Can X & Y be random but in some range?
# Q: Does Y need to equal Z?

# Analyze:
# - Run simulation over different N and check if log3(N) always achieves the desired height
# - Increase # simulations and check if the average number of messages different

# TODO:
# - Redundancy & failure

from collections import defaultdict
from collections import deque
from pptree import Node, print_tree

import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Helpers
def shrink_list(l, i1, i2):
    if i1 <= i2:
        return l[i1:i2]
    return l[i1:] + l[:i2]

def get_msg_sent(l, i, t):
    s = "[ "
    for idx, n in enumerate(l):
        if idx == i:
            s += f"({n}), "
        elif idx == t:
            s += f"*{n}*, "
        else:
            s += f"{n}, "
    return f"{s[:-2]} ]"

def agg_dicts(d1, d2):
    return {k: d1.get(k, 0) + d2.get(k, 0) for k in set(d1) | set(d2)}

def run_simulations(S, N, X, Y, Z, should_print): # S = num simulations, N = num nodes, X = 1st message, Y = 2nd message, Z = shrinkage
    # Reset global params for simulation
    global global_msg_send_counter
    global global_set_nodes_reached
    global global_map_msg_rec_counter
    global global_map_depth_reached_counter
    global global_prop_queue
    global global_missing_nodes

    # Core logic
    def prop(addr, book, depth, X, Y, Z, parent): # addr => curr_add, book => curr_addr_book, depth => curr_depth
        global global_msg_send_counter
        global global_set_nodes_reached
        global global_map_msg_rec_counter
        global global_map_depth_reached_counter
        global global_prop_queue
        global global_missing_nodes

        if len(book) == 0:
            return

        # if depth >= global_max_allowed_depth:
        #     global_map_depth_reached_counter[depth] += 1
        #     print("Max depth reached")
        #     return

        if len(global_missing_nodes) == 0:
            global_map_depth_reached_counter[depth] += 1
            return
        
        if should_print:
            print('-----') # Separator to make reading easier

        # Add node to the tree
        node = Node(addr) if parent is None else Node(addr, parent)

        # A network message was made - track it
        global_missing_nodes.discard(addr)
        global_set_nodes_reached.add(addr)
        global_map_msg_rec_counter[addr] += 1

        n = len(book)
        i = book.index(addr)
        x = (i + int(n * X)) % n
        y = (i + int(n * Y)) % n
        z = (i + int(n * Z)) % n

        x_addr = book[x]
        y_addr = book[y]
        
        if x_addr == y_addr:
            y_addr = None
        if x_addr == addr:
            x_addr = None
        
        if should_print:
            print(f"{addr} - {book}")    
            print(f"Msg 1: {get_msg_sent(book, i, x)}")
            print(f"Msg 2: {get_msg_sent(book, i, y)}")

        # Send to first target
        if x_addr is not None:
            global_msg_send_counter += 1
            x_book = book.copy()
            x_z = (x  + int(n * Z)) % n
            x_book_s = shrink_list(x_book, x, x_z)
            global_prop_queue.append((x_addr, x_book_s, depth + 1, X, Y, Z, node))

        # Send to second target
        if y_addr is not None:
            global_msg_send_counter += 1
            y_book = book.copy()
            y_z = (y  + int(n * Z)) % n
            y_book_s = shrink_list(y_book, y, y_z)
            global_prop_queue.append((y_addr, y_book_s, depth + 1, X, Y, Z, node))

        # This is a demote (not a send) so we do not increment `global_msg_send_counter`
        book_s = shrink_list(book, i, z)
        global_prop_queue.append((addr, book_s, depth + 1, X, Y, Z, node))

        return node

    ## Simulations counters
    msg_send_counter_acc = 0
    map_msg_rec_counter_acc = defaultdict(int)
    depth_counter_acc = defaultdict(int)


    # global_max_allowed_depth = 10 # log3(9) => 2 
    global_addr_book = sorted([chr(ord('A') + i) for i in range(N)])

    for _ in range(S):
        # Reset global params for simulation
        global_msg_send_counter = 0
        global_set_nodes_reached = set()
        global_map_msg_rec_counter = defaultdict(int)
        global_map_depth_reached_counter = defaultdict(int)
        global_prop_queue = deque()
        global_missing_nodes = set(global_addr_book)

        # Start simulation
        orig_addr = random.choice(global_addr_book)
        # orig_addr = global_addr_book[0]
        global_prop_queue.append((orig_addr, global_addr_book, 0, X, Y, Z, None))
        root = None
        while len(global_prop_queue) > 0:
            node = prop(*global_prop_queue.popleft())
            root = node if root is None else root

        # Print results
        if should_print:
            print('###################')
            print_tree(root, horizontal=False)    
            print(f"Target Coverage: {Y}")
            print(f"Num nodes: {N}")
            print(f"Global Send Counter: {global_msg_send_counter}")
            print(f"Global Set Reached: {sorted(list(global_set_nodes_reached))}")
            print(f"Global # Times Received: {dict(dict(sorted(global_map_msg_rec_counter.items(), key=lambda item: -item[1])))}")
            print(f"Nodes not reached: {global_set_nodes_reached.difference(global_addr_book)}")

        # Aggregate results
        depth_counter_acc = agg_dicts(depth_counter_acc, global_map_depth_reached_counter)
        msg_send_counter_acc += global_msg_send_counter
        map_msg_rec_counter_acc = agg_dicts(map_msg_rec_counter_acc, global_map_msg_rec_counter)
        
    msg_send_counter_acc /= S
    depth_counter_acc = {k:round(i/S, 3) for k,i in depth_counter_acc.items()}
    map_msg_rec_counter_acc = {k:round(i/S, 3) for k,i in map_msg_rec_counter_acc.items()}
    map_msg_rec_counter_acc = dict(sorted(map_msg_rec_counter_acc.items(), key=lambda item: -item[1]))

    return (msg_send_counter_acc, depth_counter_acc, map_msg_rec_counter_acc)

# Algo Params
S = 1000
N = 9 # Num nodes
X = 1/3 # 1st message
Y = 2/3 # 2nd Message
Z = 2/3 # Shrinkage

(msg_count, depth_acc, msg_acc) = run_simulations(S, N, X, Y, Z, True)
print("Simulation results")
print((msg_count, depth_acc, msg_acc))

-----
I - ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
Msg 1: [ A, B, *C*, D, E, F, G, H, (I) ]
Msg 2: [ A, B, C, D, E, *F*, G, H, (I) ]
-----
C - ['C', 'D', 'E', 'F', 'G', 'H']
Msg 1: [ (C), D, *E*, F, G, H ]
Msg 2: [ (C), D, E, F, *G*, H ]
-----
F - ['F', 'G', 'H', 'I', 'A', 'B']
Msg 1: [ (F), G, *H*, I, A, B ]
Msg 2: [ (F), G, H, I, *A*, B ]
-----
I - ['I', 'A', 'B', 'C', 'D', 'E']
Msg 1: [ (I), A, *B*, C, D, E ]
Msg 2: [ (I), A, B, C, *D*, E ]
-----
E - ['E', 'F', 'G', 'H']
Msg 1: [ (E), *F*, G, H ]
Msg 2: [ (E), F, *G*, H ]
-----
G - ['G', 'H', 'C', 'D']
Msg 1: [ (G), *H*, C, D ]
Msg 2: [ (G), H, *C*, D ]
-----
C - ['C', 'D', 'E', 'F']
Msg 1: [ (C), *D*, E, F ]
Msg 2: [ (C), D, *E*, F ]
-----
H - ['H', 'I', 'A', 'B']
Msg 1: [ (H), *I*, A, B ]
Msg 2: [ (H), I, *A*, B ]
-----
A - ['A', 'B', 'F', 'G']
Msg 1: [ (A), *B*, F, G ]
Msg 2: [ (A), B, *F*, G ]
-----
F - ['F', 'G', 'H', 'I']
Msg 1: [ (F), *G*, H, I ]
Msg 2: [ (F), G, *H*, I ]
-----
B - ['B', 'C', 'D', 'E']
Msg 1: [ (B), *C*,

In [2]:
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots

## Compute depth distribution
depth_l = []
for d, c in depth_acc.items():
    depth_l.extend([d] * int(c))
depths, counts = zip(*list(depth_acc.items()))

depth_mean = sum(depth_l) / len(depth_l)
depth_variance = sum([((x - depth_mean) ** 2) for x in depth_l]) / len(depth_l)
depth_std = depth_variance ** 0.5
depth_normal_data = np.random.normal(depth_mean, depth_std, size=100) # replace with your own data source

## Compute send distribution
counts_send = []
for d, c in msg_acc.items():
    counts_send.extend([d] * int(c))
nodes, counts_send = zip(*list(msg_acc.items()))

send_mean = sum(counts_send) / len(counts_send)
send_variance = sum([((x - send_mean) ** 2) for x in counts_send]) / len(counts_send)
send_std = send_variance ** 0.5
send_normal_data = np.random.normal(send_mean, send_std, size=100) # replace with your own data source

# Prepare Figure layout
fig = make_subplots(
    rows=2, cols=1,
    specs=[
        [{"secondary_y": True}],
        [{"secondary_y": True}]],
    row_heights=[5, 5],
    subplot_titles=(
        f'Depth Distribution - Nodes: {N}; Simulations: {S}; Mean: {round(depth_mean, 2)}; Std: {round(depth_std, 2)}',
        f'Message Send Distribution - Nodes: {N}; Simulations: {S}; Mean: {round(send_mean, 2)}; Std: {round(send_std, 2)}',
    ))
fig.update_layout(
    height=800,
)
fig.update_xaxes(range=[0,int(depth_mean * 2)])
fig.update_yaxes(title="Depth counter", secondary_y=False)
fig.update_yaxes(title="Depth counter - simulated normal dist", secondary_y=True)

## Show Raw Data
fig.add_trace(
    go.Bar(x=depths, y=counts, name="depth counter", opacity=0.8),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=nodes, y=counts_send, name="message send counter", opacity=0.8),
    secondary_y=False,
    row=2, col=1,
)

## Show Bars - Depths
rstd = depth_mean + depth_std
lstd = depth_mean - depth_std
fig.add_shape(type="line",x0=depth_mean, x1=depth_mean, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'darkorange', dash = 'dash'), secondary_y=True)
fig.add_shape(type="line",x0=rstd, x1=rstd, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'orangered', dash = 'dash'), secondary_y=True)
fig.add_shape(type="line",x0=lstd, x1=lstd, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'orangered', dash = 'dash'), secondary_y=True)

## Show Normal Distribution - Depths
fig2 = ff.create_distplot([depth_normal_data], ['depth counter - simulated normal dist'])
fig.add_trace(go.Histogram(fig2['data'][0], marker_color='orange', opacity=0.2), secondary_y=True)
fig.add_trace(go.Scatter(fig2['data'][1], line=dict(color='orange', width=0.5) ), secondary_y=True)
fig.add_trace(go.Scatter(fig2['data'][2], line=dict(color='orange', width=0.5)), secondary_y=True)

## Print Results
print(f"Target Coverage: {round(Y, 3)}")
print(f"Num nodes: {N}")
print(f"Num simulations: {S}")
print(f"Avg # messages per simulation: {round(msg_count / S, 2)}")
fig.show()

Target Coverage: 0.667
Num nodes: 9
Num simulations: 1000
Avg # messages per simulation: 0.02


In [3]:
def plot(map_to_msg_count, map_to_depth_count, var_name):
    # Prepare Figure layout
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{}, {}]],
        row_heights=[5],
        subplot_titles=(
            f'Avg # Msgs / Node',
            f'Avg Max Depth to Full Coverage',
        ))
    fig.update_layout(
        height=500,
    )
    # fig.update_xaxes(range=[0,int(depth_mean * 2)])
    fig.update_yaxes(title="# Msgs / Node", row=1, col=1)
    fig.update_xaxes(title=var_name, row=1, col=1)

    fig.update_yaxes(title="Max Depth", row=1, col=2)
    fig.update_xaxes(title=var_name, row=1, col=2)

    # ## Show Raw Data
    (x, y) = zip(*dict(map_to_msg_count).items())
    fig.add_trace(
        go.Scatter(x=x, y=y, name="msg count", opacity=0.8),
        row=1, col=1
    )
    (x, y) = zip(*dict(map_to_depth_count).items())
    fig.add_trace(
        go.Scatter(x=x, y=y, name="msg count", opacity=0.8),
        row=1, col=2
    )
    fig.show()

def single_iteration(var, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count):
    try:
        (_, depth_acc, msg_acc) = run_simulations(S, N, X, Y, Z, False)
        # print("Simulation results")
        # print((msg_count, depth_acc, msg_acc))

        # Message Count
        avg_msg = sum(msg_acc.values()) / len(msg_acc)
        map_to_msg_count[var] = avg_msg

        # Depth
        num_total = 0
        denom_total = 0
        for d, c in depth_acc.items():
            num_total += c * d
            denom_total += c
        avg_depth = num_total / denom_total
        map_to_depth_count[var] = avg_depth
    except Exception as e:
        return


In [4]:

S = 1000
X = 1/3 # 1st message
Y = 2/3 # 2nd Message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for N in range(3, 81):
    single_iteration(N, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "# Nodes")    

In [5]:
import numpy as np

S = 1000
N = 27
X = 1/3 # 1st message
Y = 3/4 # 2nd Message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for X in np.arange(0.1, Y, 0.01):
    print("X:", X)
    single_iteration(X, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "X target %")

X: 0.1
X: 0.11
X: 0.12
X: 0.13
X: 0.13999999999999999
X: 0.14999999999999997
X: 0.15999999999999998
X: 0.16999999999999998
X: 0.17999999999999997
X: 0.18999999999999995
X: 0.19999999999999996
X: 0.20999999999999996
X: 0.21999999999999995
X: 0.22999999999999995
X: 0.23999999999999994
X: 0.24999999999999992
X: 0.2599999999999999
X: 0.2699999999999999
X: 0.2799999999999999
X: 0.2899999999999999
X: 0.29999999999999993
X: 0.30999999999999994
X: 0.3199999999999999
X: 0.32999999999999985
X: 0.33999999999999986
X: 0.34999999999999987
X: 0.3599999999999999
X: 0.3699999999999999
X: 0.3799999999999999
X: 0.3899999999999999
X: 0.3999999999999998
X: 0.4099999999999998
X: 0.4199999999999998
X: 0.4299999999999998
X: 0.43999999999999984
X: 0.44999999999999984
X: 0.45999999999999985
X: 0.46999999999999986
X: 0.47999999999999976
X: 0.48999999999999977
X: 0.4999999999999998
X: 0.5099999999999998
X: 0.5199999999999998
X: 0.5299999999999998
X: 0.5399999999999998
X: 0.5499999999999998
X: 0.5599999999999997


In [6]:
import numpy as np

S = 1000
N = 27
X = 0.25 # 1st message
Y = 0.75 # 2nd Message

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for Z in np.arange(0.1, 0.9, 0.01):
    print("Z:", Z)
    single_iteration(Z, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "Shrinkage %")

Z: 0.1
Z: 0.11
Z: 0.12
Z: 0.13
Z: 0.13999999999999999
Z: 0.14999999999999997
Z: 0.15999999999999998
Z: 0.16999999999999998
Z: 0.17999999999999997
Z: 0.18999999999999995
Z: 0.19999999999999996
Z: 0.20999999999999996
Z: 0.21999999999999995
Z: 0.22999999999999995
Z: 0.23999999999999994
Z: 0.24999999999999992
Z: 0.2599999999999999
Z: 0.2699999999999999
Z: 0.2799999999999999
Z: 0.2899999999999999
Z: 0.29999999999999993
Z: 0.30999999999999994
Z: 0.3199999999999999
Z: 0.32999999999999985
Z: 0.33999999999999986
Z: 0.34999999999999987
Z: 0.3599999999999999
Z: 0.3699999999999999
Z: 0.3799999999999999
Z: 0.3899999999999999
Z: 0.3999999999999998
Z: 0.4099999999999998
Z: 0.4199999999999998
Z: 0.4299999999999998
Z: 0.43999999999999984
Z: 0.44999999999999984
Z: 0.45999999999999985
Z: 0.46999999999999986
Z: 0.47999999999999976
Z: 0.48999999999999977
Z: 0.4999999999999998
Z: 0.5099999999999998
Z: 0.5199999999999998
Z: 0.5299999999999998
Z: 0.5399999999999998
Z: 0.5499999999999998
Z: 0.5599999999999997


In [8]:
import numpy as np

S = 1000
N = 27
X = 0.01 # 1st message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for Y in np.arange(X, 
0.9, 0.01):
    print("Y:", Y)
    single_iteration(Y, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "Second Target %")

Y: 0.01
Y: 0.02
Y: 0.03
Y: 0.04
Y: 0.05
Y: 0.060000000000000005
Y: 0.06999999999999999
Y: 0.08
Y: 0.09
Y: 0.09999999999999999
Y: 0.11
Y: 0.12
Y: 0.13
Y: 0.14
Y: 0.15000000000000002
Y: 0.16
Y: 0.17
Y: 0.18000000000000002
Y: 0.19
Y: 0.2
Y: 0.21000000000000002
Y: 0.22
Y: 0.23
Y: 0.24000000000000002
Y: 0.25
Y: 0.26
Y: 0.27
Y: 0.28
Y: 0.29000000000000004
Y: 0.3
Y: 0.31
Y: 0.32
Y: 0.33
Y: 0.34
Y: 0.35000000000000003
Y: 0.36000000000000004
Y: 0.37
Y: 0.38
Y: 0.39
Y: 0.4
Y: 0.41000000000000003
Y: 0.42000000000000004
Y: 0.43
Y: 0.44
Y: 0.45
Y: 0.46
Y: 0.47000000000000003
Y: 0.48000000000000004
Y: 0.49
Y: 0.5
Y: 0.51
Y: 0.52
Y: 0.53
Y: 0.54
Y: 0.55
Y: 0.56
Y: 0.5700000000000001
Y: 0.5800000000000001
Y: 0.59
Y: 0.6
Y: 0.61
Y: 0.62
Y: 0.63
Y: 0.64
Y: 0.65
Y: 0.66
Y: 0.67
Y: 0.68
Y: 0.6900000000000001
Y: 0.7000000000000001
Y: 0.7100000000000001
Y: 0.72
Y: 0.73
Y: 0.74
Y: 0.75
Y: 0.76
Y: 0.77
Y: 0.78
Y: 0.79
Y: 0.8
Y: 0.81
Y: 0.8200000000000001
Y: 0.8300000000000001
Y: 0.8400000000000001
Y: 0.85
Y: 